# MABe Challenge - XGBoost inference Notebook

📝 **Note:** Please note that comments and explanations are in Japanese. However, I've made an effort to write clear, self-explanatory code that should be accessible to non-Japanese speakers as well.

## training notebook: 
https://www.kaggle.com/code/hutch1221/mabe-starter-train-ja/notebook

In [ ]:
!pip install -q --no-index --find-links=/kaggle/input/mabe-package xgboost==3.1.1

In [ ]:
!cp /kaggle/input/mabe-starter-train-ja/self_features.py .
!cp /kaggle/input/mabe-starter-train-ja/pair_features.py .
!cp /kaggle/input/mabe-starter-train-ja/robustify.py .
!cp -r /kaggle/input/mabe-starter-train-ja/results .

In [ ]:
import gc
import itertools
import re
import sys
from collections import defaultdict
from pathlib import Path

import joblib
import numpy as np
import pandas as pd
import polars as pl
import xgboost as xgb
from tqdm.auto import tqdm

sys.path.append("/kaggle/usr/lib/mabe-f-beta")
from metric import score

In [ ]:
# const
INPUT_DIR = Path("/kaggle/input/MABe-mouse-behavior-detection")
TRAIN_TRACKING_DIR = INPUT_DIR / "train_tracking"
TRAIN_ANNOTATION_DIR = INPUT_DIR / "train_annotation"
TEST_TRACKING_DIR = INPUT_DIR / "test_tracking"

WORKING_DIR = Path("/kaggle/working")

INDEX_COLS = [
    "video_id",
    "agent_mouse_id",
    "target_mouse_id",
    "video_frame",
]

BODY_PARTS = [
    "ear_left",
    "ear_right",
    "nose",
    "neck",
    "body_center",
    "lateral_left",
    "lateral_right",
    "hip_left",
    "hip_right",
    "tail_base",
    "tail_tip",
]

SELF_BEHAVIORS = [
    "biteobject",
    "climb",
    "dig",
    "exploreobject",
    "freeze",
    "genitalgroom",
    "huddle",
    "rear",
    "rest",
    "run",
    "selfgroom",
]

PAIR_BEHAVIORS = [
    "allogroom",
    "approach",
    "attack",
    "attemptmount",
    "avoid",
    "chase",
    "chaseattack",
    "defend",
    "disengage",
    "dominance",
    "dominancegroom",
    "dominancemount",
    "ejaculate",
    "escape",
    "flinch",
    "follow",
    "intromit",
    "mount",
    "reciprocalsniff",
    "shepherd",
    "sniff",
    "sniffbody",
    "sniffface",
    "sniffgenital",
    "submit",
    "tussle",
]

In [ ]:
import gc
import re
import ast
import sys
import warnings
import itertools
from pathlib import Path
from dataclasses import dataclass
from typing import Dict, List, Any, Optional

import numpy as np
import pandas as pd
from scipy.ndimage import gaussian_filter1d
from tqdm.auto import tqdm


warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
np.seterr(invalid="ignore", divide="ignore")

@dataclass
class FeatureConfig:
    fps: float = 30.0
    pix_per_cm: float = 1.0
    smooth_sigma: float = 1.0
    use_pairwise: bool = True

@dataclass
class AgentContext:
    idx: pd.Index
    pos: np.ndarray
    vel: np.ndarray
    speed: np.ndarray
    acc: np.ndarray
    cx: pd.Series
    cy: pd.Series
    speed_series: pd.Series
    raw_df: Optional[pd.DataFrame] = None

class FeatureExtractor:
    """
    Class trích xuất đặc trưng hành vi từ dữ liệu tracking.
    """
    def __init__(self, fps: float, pix_per_cm: float, smooth_sigma: float = 1.0, use_pairwise: bool = True):
        # Map tham số từ init vào Config
        self.cfg = FeatureConfig(
            fps=float(fps), 
            pix_per_cm=float(pix_per_cm), 
            smooth_sigma=smooth_sigma,
            use_pairwise=use_pairwise
        )
        
        # Đăng ký các hàm feature sẽ chạy
        self.feature_registry = {
            "kinematics": self._feat_basic_kinematics,
            "multiscale": self._feat_multiscale,
            "long_range": self._feat_long_range,
            "cumulative": self._feat_cumulative,
            "curvature": self._feat_curvature,
            "speed_asym": self._feat_speed_asym,
            "gauss_shift": self._feat_gauss_shift,
            "grooming": self._feat_avoidance_trajectory,
            "pose": self._feat_pose_shape,
            "a": self._feat_submission_temporal,
            "pose_adv": self._feat_pose_advanced, 
            "pairwise": self._feat_pairwise
        }

    # --- Helpers ---
    def _scale(self, n_frames_30fps: int) -> int:
        """Quy đổi số frame từ chuẩn 30fps sang fps thực tế của video."""
        return max(1, int(round(n_frames_30fps * self.cfg.fps / 30.0)))

    def _to_cm(self, arr):
        """Chuyển pixel -> cm."""
        return arr / self.cfg.pix_per_cm

    def _smooth(self, x):
        """Làm mượt dữ liệu bằng Gaussian filter."""
        if self.cfg.smooth_sigma is None or x.shape[0] < 3: return x
        if np.all(np.isnan(x)): return x
        return gaussian_filter1d(x, sigma=self.cfg.smooth_sigma, axis=0, mode="nearest")

    def _forward_fill_nan(self, pos):
        """
        Điền dữ liệu thiếu (NaN) bằng giá trị hợp lệ trước đó (Forward Fill).
        """
        if np.all(np.isnan(pos)):
            return np.zeros_like(pos)

        pos_ffill = pos.copy()
        mask = np.any(~np.isnan(pos_ffill), axis=1)
        if not mask.any():
            return np.zeros_like(pos_ffill)

        valid_idx = np.where(mask)[0]
        first, last = valid_idx[0], valid_idx[-1]
        pos_ffill[:first] = pos_ffill[first]
        pos_ffill[last + 1:] = pos_ffill[last]
        df_temp = pd.DataFrame(pos_ffill)
        df_temp = df_temp.ffill()
        return df_temp.to_numpy()
    
    def _speed_series(self, cx: pd.Series, cy: pd.Series) -> pd.Series:
        dx = cx.diff()
        dy = cy.diff()
        v = np.hypot(dx, dy).fillna(0.0) * self.cfg.fps
        return v.astype("float32")
    
    def _roll_future_mean(self, s: pd.Series, w: int, min_p: int = 1) -> pd.Series:
        return s.iloc[::-1].rolling(w, min_periods=min_p).mean().iloc[::-1]

    def _roll_future_var(self, s: pd.Series, w: int, min_p: int = 2) -> pd.Series:
        return s.iloc[::-1].rolling(w, min_periods=min_p).var().iloc[::-1]

    # --- Core Logic ---
    def _compute_kinematics(self, pos_px: np.ndarray):
        """
        Tính toán vật lý cơ bản: Pos(cm), Vel, Speed, Acc.
        Input: Array [Frames, 2] (pixel).
        Output: Tuple (pos_cm, vel, speed, acc).
        """
        pos_ffill = self._forward_fill_nan(pos_px)
        pos_cm = self._to_cm(pos_ffill.astype(np.float32))
        pos_cm = self._smooth(pos_cm)                                               # [F, 2]

        dt = 1.0 / self.cfg.fps
        vel = np.zeros_like(pos_cm, dtype=np.float32)
        vel[1:] = (pos_cm[1:] - pos_cm[:-1]) / dt                                   # [F, 2: (vx, vy)]
        speed = np.linalg.norm(vel, axis=1, keepdims=True).astype(np.float32)       # [F, 1]

        acc = np.zeros_like(pos_cm, dtype=np.float32)                          
        acc[1:] = (vel[1:] - vel[:-1]) / dt                                         # [F, 2:(ax, ay)]
        return pos_cm.astype(np.float32), vel, speed, acc

    def _build_context(self, frames, pos_px, mouse_df=None) -> AgentContext:
        """
        Tạo AgentContext chứa đầy đủ thông tin vật lý của 1 con chuột.
        """
        p, v, s, a = self._compute_kinematics(pos_px)
        idx = pd.Index(frames, name="frame")
        
        return AgentContext(
            idx=idx, pos=p, vel=v, speed=s, acc=a, 
            cx=pd.Series(p[:, 0], index=idx), 
            cy=pd.Series(p[:, 1], index=idx), 
            speed_series=pd.Series(s[:, 0], index=idx), 
            raw_df=mouse_df
        )

    # --- Feature Modules ---
    def _feat_basic_kinematics(self, ctx: AgentContext, **kwargs) -> Dict:
        """
        Lấy các giá trị thô: tọa độ x, y, vận tốc vx, vy, tốc độ, gia tốc ax, ay.
        """
        return {
            "a_x": ctx.pos[:, 0], "a_y": ctx.pos[:, 1],
            "a_vx": ctx.vel[:, 0], "a_vy": ctx.vel[:, 1],
            "a_speed": ctx.speed[:, 0],
            "a_ax": ctx.acc[:, 0], "a_ay": ctx.acc[:, 1]
        }

    def _feat_multiscale(self, ctx: AgentContext, **kwargs) -> Dict:
        """
        Tính tốc độ trung bình (Mean) và độ lệch chuẩn (Std) ở đa mức thời gian.
        Feature 'sp_ratio' đo độ bùng nổ (Burstiness).
        """
        feats = {}
        speed = ctx.speed_series
        frame_scales = [10, 40, 160]
        for scale in frame_scales:
            ws = self._scale(scale)
            if len(speed) >= ws:
                roller = speed.rolling(ws, min_periods=max(1, ws//4), center=True)
                feats[f"sp_m{scale}"] = roller.mean().astype("float32")
                feats[f"sp_s{scale}"] = roller.std().astype("float32")
        feats[f"sp_ratio"] = feats["sp_m10"] / (feats["sp_m160"] + 1e-6)
        return feats 
        
    def _feat_long_range(self, ctx: AgentContext, **kwargs) -> Dict:
        """
        Đặc trưng ngữ cảnh dài hạn:
        - x_ml, y_ml: Vị trí trung bình trong quá khứ.
        - sp_pct: Xếp hạng (percentile) của tốc độ hiện tại so với quá khứ.
        """
        feats: Dict[str, pd.Series] = {}
        speed = ctx.speed_series

        for window in [120, 240]:
            ws = self._scale(window)
            if len(ctx.cx) >= ws:
                feats[f"x_ml{window}"] = ctx.cx.rolling(ws, min_periods=max(5, ws // 6), center=True).mean()
                feats[f"y_ml{window}"] = ctx.cy.rolling(ws, min_periods=max(5, ws // 6), center=True).mean()

        for span in [60, 120]:
            s = self._scale(span)
            feats[f"x_e{span}"] = ctx.cx.ewm(span=s, min_periods=1).mean()
            feats[f"y_e{span}"] = ctx.cy.ewm(span=s, min_periods=1).mean()

        for window in [60, 120]:
            ws = self._scale(window)
            if len(speed) >= ws:
                feats[f"sp_pct{window}"] = speed.rolling(
                    ws, min_periods=max(5, ws // 6), center=True
                ).rank(pct=True)
        return feats
    

    def _feat_curvature(self, ctx: AgentContext, **kwargs) -> Dict:
        feats = {}

        vel_x, vel_y = ctx.vel[:, 0], ctx.vel[:, 1]
        acc_x, acc_y = ctx.acc[:, 0], ctx.acc[:, 1]
        cross_prod = vel_x * acc_y - vel_y * acc_x
        vel_mag = np.sqrt(vel_x**2 + vel_y**2)
        moving_mask = vel_mag > 2.0
        vel_mag_safe = np.maximum(vel_mag, 0.1 / self.cfg.fps)
        raw_curv = cross_prod / (vel_mag_safe**3)
        raw_curv = np.where(moving_mask, raw_curv, 0.0)
        min_turn_radius_cm = 0.5
        max_k = 1.0 / min_turn_radius_cm
        raw_curv = np.clip(raw_curv, -max_k, max_k)
        abs_curv = np.abs(raw_curv)
        abs_curv_series = pd.Series(abs_curv, index=ctx.idx)

        for w in [30, 60]:
            ws = self._scale(w)
            min_p = max(ws // 3, 1)
            feats[f"curv_mean_{w}"] = abs_curv_series.rolling(ws, min_periods=min_p).mean()

        angle = np.arctan2(vel_y, vel_x)
        angle_series = pd.Series(angle, index=ctx.idx)
        angle_change = np.abs(angle_series.diff().fillna(0.0))
        angle_change = np.where(angle_change > np.pi, 2 * np.pi - angle_change, angle_change)
        angle_change_series = pd.Series(angle_change, index=ctx.idx)
        angle_change_series = pd.Series(np.where(moving_mask, angle_change_series, 0.0), index=ctx.idx)

        ws = self._scale(30)
        feats["turn_rate_30"] = angle_change_series.rolling(ws, min_periods=max(ws // 3, 1)).sum()

        return feats
    
    def _feat_cumulative(self, ctx: AgentContext, **kwargs) -> Dict:
        """
        Tổng quãng đường di chuyển trong một khoảng thời gian dài xung quanh frame hiện tại.
        """
        feats = {}
        L = max(1, self._scale(180))
        step = np.hypot(ctx.cx.diff(), ctx.cy.diff()).fillna(0.0)
        path = step.rolling(2 * L + 1, min_periods=max(5, L // 6), center=True).sum()
        feats["path_cum180"] =  path.fillna(0.0).astype("float32")
        return feats

    def _feat_speed_asym(self, ctx: AgentContext, **kwargs) -> Dict:
        """
        Bất đối xứng tốc độ (Tương lai - Quá khứ).
        """
        w = max(3, self._scale(30))
        v = ctx.speed_series
        v_past = v.rolling(w, min_periods=1).mean()
        v_fut = self._roll_future_mean(v, w, min_p=1)
        return {"spd_asym_1s": (v_fut - v_past).fillna(0.0)}
    
    def _feat_gauss_shift(self, ctx: AgentContext, **kwargs) -> Dict:
        """
        Độ lệch Gaussian (KL Divergence) giữa quá khứ và tương lai.
        Đo lường sự thay đổi trạng thái thống kê.
        """
        w = max(5, self._scale(30))
        v = ctx.speed_series
        mu_p = v.rolling(w, min_periods=1).mean()
        va_p = v.rolling(w, min_periods=1).var().clip(lower=1e-6)
        mu_f = self._roll_future_mean(v, w, min_p=1)
        va_f = self._roll_future_var(v, w, min_p=1).clip(lower=1e-6)

        kl_pf = 0.5 * (
            (va_p / va_f) + ((mu_f - mu_p) ** 2) / va_f - 1.0 + np.log(va_f / va_p)
        )
        kl_fp = 0.5 * (
            (va_f / va_p) + ((mu_p - mu_f) ** 2) / va_p - 1.0 + np.log(va_p / va_f)
        )
        return {
            "spd_symkl_1s": (kl_pf + kl_fp).replace([np.inf, -np.inf], np.nan).fillna(0.0)
        }

    def _feat_avoidance_trajectory(self, ctx: AgentContext, target_ctx: AgentContext = None, **kwargs) -> Dict[str, pd.Series]:
        """
        Tính toán quỹ đạo né tránh:
        1. Relative Heading: Góc di chuyển so với hướng tới đối thủ.
        2. Future Distance Gain: Dự báo xem hành động này có giúp chuột ra xa đối thủ trong tương lai không.
        """
        feats = {}
        if target_ctx is None: 
            return feats

        idx = ctx.idx
        def zero(): return pd.Series(0.0, index=idx, dtype="float32")

        # --- 1. RELATIVE HEADING (Góc lệch hướng đi) ---
        # Vector từ Tôi -> Địch
        rel_vec = target_ctx.pos - ctx.pos
        # Góc hướng tới địch (Angle to Target)
        angle_to_target = np.arctan2(rel_vec[:, 1], rel_vec[:, 0])
        
        # Góc di chuyển của Tôi (My Heading)
        my_heading = np.arctan2(ctx.vel[:, 1], ctx.vel[:, 0])
        
        # Độ lệch góc (Absolute Difference)
        # Cần xử lý wrap góc (ví dụ: lệch giữa 179 độ và -179 độ là 2 độ chứ ko phải 358)
        diff = np.abs(angle_to_target - my_heading)
        diff = np.minimum(diff, 2*np.pi - diff) # Chuẩn hóa về [0, pi]
        
        # Feature: Cosine của góc lệch
        # 1.0 (0 độ) -> Lao vào
        # 0.0 (90 độ) -> AVOID (Lách ngang)
        # -1.0 (180 độ) -> Escape
        feats["heading_rel_cos"] = pd.Series(np.cos(diff), index=idx, dtype="float32")
        
        # Feature: Góc lệch tuyệt đối (đổi ra độ cho dễ hình dung nếu cần, ở đây để rad)
        feats["heading_rel_abs"] = pd.Series(diff, index=idx, dtype="float32")


        # --- 2. FUTURE DISTANCE GAIN (Hiệu quả tránh né) ---
        # "Sau 15 frame (0.5s) hoặc 30 frame (1s), mình có xa nó ra không?"
        
        dist_now = np.linalg.norm(rel_vec, axis=1)
        s_dist = pd.Series(dist_now, index=idx)
        
        scales = [15, 30] # 0.5s và 1s
        for w in scales:
            ws = self._scale(w)
            
            # Lấy khoảng cách ở tương lai (shift ngược lên)
            # s.shift(-ws) là giá trị của t + ws
            dist_future = s_dist.shift(-ws)
            gain = dist_future - s_dist
            
            feats[f"dist_gain_{w}f"] = gain.fillna(0.0).astype("float32")

        return feats
    
    def _extract_part(self, ctx: AgentContext, part: str) -> Optional[np.ndarray]:
        if ctx.raw_df is None: return None
        if part not in ctx.raw_df.columns.get_level_values(0): return None
        try:
            sub_df = ctx.raw_df.xs(part, axis=1, level=0)[["x", "y"]].reindex(ctx.idx)
        except KeyError: return None
        raw = sub_df.to_numpy()
        raw = self._forward_fill_nan(raw)
        cm = self._to_cm(raw.astype(np.float32))
        return self._smooth(cm)
    
    def _extract_parts_dict(self, ctx: AgentContext, parts: List[str] = None) -> Dict[str, Optional[np.ndarray]]:
        out = {}
        for p in parts:
            out[p] = self._extract_part(ctx, p)
        return out
        
    def _feat_pose_shape(self, ctx: AgentContext, **kwargs) -> Dict:
        """
        Placeholder cho các đặc trưng hình dáng (Elongation, Body Angle...).
        """
        feats = {}

        def zero(): return pd.Series(0.0, index=ctx.idx, dtype="float32")

        def dist(k1, k2):
            p1, p2 = parts.get(k1), parts.get(k2)
            if p1 is None or p2 is None: return zero()
            d = np.linalg.norm(p1 - p2, axis=1)
            return pd.Series(d, index=ctx.idx, dtype="float32")
        
        def body_angle():
            if parts.get("nose") is None: return zero()
            if parts.get("body_center") is None: return zero()
            if parts.get("tail_base") is None: return zero()

            v1 = parts.get("nose") - parts.get("body_center")
            v2 = parts.get("tail_base") - parts.get("body_center")
            dot_product = np.sum(v1 * v2, axis=1)
            mag = np.linalg.norm(v1, axis=1) * np.linalg.norm(v2, axis=1)
            cos_angle = np.clip(dot_product / (mag + 1e-6), -1.0, 1.0).astype("float32")
            return cos_angle
        
        def elongation():
            if parts.get("nose")          is None: return zero()
            if parts.get("tail_base")     is None: return zero()
            if parts.get("lateral_left")  is None: return zero()
            if parts.get("lateral_right") is None: return zero()

            d1 = dist("nose", "tail_base")
            d2 = dist("lateral_left", "lateral_right")
            elongation = d1 / (d2 + 1e-6).astype("float32")
            return elongation
        
        def vel(part: str, n_frames_30fps: int) -> Dict:
            part_pos = self._extract_part(ctx, part)
            if part_pos is None: return zero()
            
            s_x = pd.Series(part_pos[:, 0], index=ctx.idx)
            s_y = pd.Series(part_pos[:, 1], index=ctx.idx)
            raw_speed = self._speed_series(s_x, s_y)

            ws = self._scale(n_frames_30fps)
            val = raw_speed.rolling(ws, min_periods=1, center=True).mean()
            return val.astype("float32")


        target_parts = ["nose", "neck", "body_center", "tail_base", 
                        "ear_left", "ear_right", 
                        "lateral_left", "lateral_right", "tail_midpoint", "tail_tip"]
        
        parts = self._extract_parts_dict(ctx, target_parts)

        feats["a_body_width"]                = dist("lateral_left", "lateral_right")
        feats["aa_nose_bodycenter_dist"]     = dist("nose", "body_center")
        feats["aa_nose_tailbase_dist"]       = dist("nose", "tail_base")
        feats["aa_bodycenter_tailbase_dist"] = dist("body_center", "tail_base")
        
        feats["aa_bodycenter_ear_l_dist"]    = dist("body_center", "ear_left")
        feats["aa_bodycenter_ear_r_dist"]    = dist("body_center", "ear_right")
        feats["aa_bodycenter_lateral_l_dist"]= dist("body_center", "lateral_left")
        feats["aa_bodycenter_lateral_r_dist"]= dist("body_center", "lateral_right")
        
        feats["a_body_angle"]                = body_angle()
        feats["a_elongation"]                = elongation()
        feats["a_tail_base_vel_500ms"]       = vel("tail_base", 15)
        feats["a_tail_base_vel_1000ms"]      = vel("tail_base", 30)
        feats["a_tail_base_vel_2000ms"]      = vel("tail_base", 60)
        feats["a_tail_base_vel_3000ms"]      = vel("tail_base", 90)
        feats["a_nose_vel_500ms"]            = vel("nose", 15)
        feats["a_nose_vel_1000ms"]           = vel("nose", 30)
        feats["a_nose_vel_2000ms"]           = vel("nose", 60)
        feats["a_nose_vel_3000ms"]           = vel("nose", 90)
        feats["a_ear_right_vel_500ms"]       = vel("ear_right", 15)
        feats["a_ear_right_vel_1000ms"]      = vel("ear_right", 30)
        feats["a_ear_right_vel_2000ms"]      = vel("ear_right", 60)
        feats["a_ear_right_vel_3000ms"]      = vel("ear_right", 90)
        # len_1 = dist("tail_base", "tail_midpoint")
        # len_2 = dist("tail_midpoint", "tail_tip")
        # len_full = dist("tail_base", "tail_tip")
        # feats["tail_curl"] = ((len_1 + len_2) / (len_full + 1e-6)).astype("float32")
        return feats

    def _feat_submission_temporal(self, ctx: AgentContext, target_ctx: AgentContext = None, **kwargs) -> Dict[str, pd.Series]:
        """
        Đặc trưng 'Ký ức sợ hãi' (Fear Memory) để bắt Submit tĩnh.
        Giúp phân biệt Submit (sau khi bị đánh) vs Rest (bình yên).
        """
        feats = {}
        if target_ctx is None: return feats
        
        idx = ctx.idx
        
        # --- 1. XÂY DỰNG TÍN HIỆU XUNG ĐỘT GỐC (RAW CONFLICT SIGNAL) ---
        # Conflict = (Nó nhanh) * (Nó hướng về tôi) * (Ở gần)
        
        # A. Nó hướng về tôi không? (Gaze Cosine)
        # Vector nối Tôi -> Nó
        vec_to_target = target_ctx.pos - ctx.pos
        dist = np.linalg.norm(vec_to_target, axis=1)
        dist_safe = pd.Series(dist, index=idx).replace(0, 1e-6)
        
        # Vector vận tốc của Nó
        t_vel = target_ctx.vel
        
        # Dot product: Vận tốc Nó . Vector hướng về Tôi (Ngược dấu với vec_to_target)
        # vec_target_to_me = -vec_to_target
        # dot > 0 nghĩa là nó đang lao về phía tôi
        dot_threat = np.sum(t_vel * (-vec_to_target), axis=1)
        
        # Threat Score tức thời (cm/s hướng về nạn nhân)
        # Chỉ tính khi nó lại gần (< 15cm)
        threat_raw = (dot_threat / dist_safe).clip(lower=0) 
        threat_raw = threat_raw * (dist_safe < 15.0).astype(float)
        threat_series = pd.Series(threat_raw, index=idx, dtype="float32")

        # --- 2. KÝ ỨC SỢ HÃI (FEAR MEMORY - QUAN TRỌNG NHẤT) ---
        # Dùng Rolling Max để "kéo dài" nỗi sợ.
        # Nếu 2 giây trước nó lao vào tôi, thì giờ tôi vẫn đang sợ.
        
        # Cửa sổ 3 giây (90 frames)
        ws_memory = self._scale(90)
        
        # Fear Level = Max threat trong 3 giây qua
        feats["fear_memory_3s"] = threat_series.rolling(ws_memory, min_periods=1).max().astype("float32")

        # --- 3. TRẠNG THÁI SUBMIT (KẾT HỢP) ---
        # Submit = (Tôi đang đứng yên) * (Tôi đang co cụm) * (Tôi đang sợ)
        
        # Tôi đứng yên (< 1 cm/s)
        my_speed = ctx.speed_series
        is_still = (my_speed < 1.0).astype(float)
        
        # Tôi co cụm (Dùng a_elongation thấp hoặc body_width/length cao)
        # Giả sử bạn đã tính a_elongation ở hàm pose (thấp là co cụm)
        # Nếu chưa có thì dùng tạm logic: elongation < 1.2
        # Ở đây mình tạo feature giả lập độ co cụm nếu chưa có
        parts = self._extract_parts_dict(ctx, ["nose", "tail_base"])
        if parts["nose"] is not None:
            spine_len = np.linalg.norm(parts["nose"] - parts["tail_base"], axis=1)
            is_compact = (spine_len < 8.0).astype(float) # Ví dụ chuột dài < 8cm là co
            is_compact = pd.Series(is_compact, index=idx)
        else:
            is_compact = pd.Series(0.0, index=idx)

        # FINAL SCORE
        # Đây là feature định danh cho Submit tĩnh
        feats["static_submit_prob"] = (
            is_still * is_compact * feats["fear_memory_3s"]
        ).astype("float32")

        return feats

    def _feat_pose_advanced(self, ctx: AgentContext, **kwargs) -> Dict[str, pd.Series]:
        feats = {}
        idx = ctx.idx
        def zero(): return pd.Series(0.0, index=idx, dtype="float32")
        
        # 1. TRÍCH XUẤT DỮ LIỆU
        # Nhóm cơ bản
        parts = self._extract_parts_dict(ctx, ["nose", "neck", "body_center", "tail_base", 
                                               "lateral_left", "lateral_right", 
                                               "tail_midpoint", "tail_tip"])
        
        # Nhóm Headpiece (Lấy đại diện 4 góc trên để tính toán cho nhẹ)
        hp_parts = self._extract_parts_dict(ctx, [
            "headpiece_topfrontleft", "headpiece_topfrontright",
            "headpiece_topbackleft", "headpiece_topbackright"
        ])

        # Helper tính khoảng cách
        def dist(p1, p2):
            if p1 is None or p2 is None: return zero()
            return pd.Series(np.linalg.norm(p1 - p2, axis=1), index=idx, dtype="float32")
        
        # 2. Spine Length (Neck -> TailBase) - Key cho Rear
        feats["a_spine_len"] = dist(parts["neck"], parts["tail_base"])

        # --- B. HEADPIECE ORIENTATION (Góc nhìn 3D) ---
        # Đây là feature độc quyền của dataset này
        
        fl = hp_parts["headpiece_topfrontleft"]
        fr = hp_parts["headpiece_topfrontright"]
        bl = hp_parts["headpiece_topbackleft"]
        br = hp_parts["headpiece_topbackright"]
        
        if fl is not None and br is not None:
            # 1. Head Length 2D (Front to Back)
            # Trung điểm Front
            front_mid = (fl + fr) / 2
            back_mid = (bl + br) / 2
            head_len_2d = np.linalg.norm(front_mid - back_mid, axis=1)
            
            # Nếu head_len_2d ngắn lại -> Đầu đang ngẩng lên hoặc cúi xuống (Pitch)
            # (Giả sử chiều dài thật của headpiece là hằng số Max quan sát được)
            feats["head_pitch_proxy"] = pd.Series(head_len_2d, index=idx, dtype="float32")
            
            # 2. Head Roll (Độ nghiêng)
            # So sánh đường chéo: (FrontLeft-BackRight) vs (FrontRight-BackLeft)
            d1 = np.linalg.norm(fl - br, axis=1)
            d2 = np.linalg.norm(fr - bl, axis=1)
            feats["head_roll_proxy"] = pd.Series(np.abs(d1 - d2), index=idx, dtype="float32")
        else:
            feats["head_pitch_proxy"] = zero()
            feats["head_roll_proxy"] = zero()

        # --- C. TAIL DYNAMICS (Đuôi) ---
        
        # 1. Tail Straightness (Đuôi thẳng hay cong?)
        # (Base->Mid) + (Mid->Tip)
        len_1 = dist(parts["tail_base"], parts["tail_midpoint"])
        len_2 = dist(parts["tail_midpoint"], parts["tail_tip"])
        len_full = dist(parts["tail_base"], parts["tail_tip"])
        
        # Ratio: Nếu = 1.0 là thẳng tắp. Lớn hơn 1.0 là cong.
        feats["tail_curl"] = ((len_1 + len_2) / (len_full + 1e-6)).astype("float32")
        
        # 2. Tail Tip Speed (Độ quất đuôi - Aggression)
        # Tính riêng vận tốc mũi đuôi
        if parts["tail_tip"] is not None:
            tip_pos = parts["tail_tip"]
            s_x = pd.Series(tip_pos[:, 0], index=idx)
            s_y = pd.Series(tip_pos[:, 1], index=idx)
            # Lấy rung động tần số cao (cửa sổ nhỏ 15 frames)
            raw_speed = self._speed_series(s_x, s_y)
            feats["tail_tip_speed_fast"] = raw_speed.rolling(self._scale(15), center=True).mean().astype("float32")
        else:
            feats["tail_tip_speed_fast"] = zero()

        return feats


    def _feat_pairwise(self, ctx: AgentContext, target_ctx: AgentContext = None, **kwargs) -> Dict:
        """
        Đặc trưng tương tác cặp đôi (Pairwise): Khoảng cách, Tốc độ tiếp cận.
        """
        feats: Dict[str, pd.Series] = {}
        if target_ctx is None: 
            return feats

        idx = ctx.idx
        def zero(): return pd.Series(0.0, index=idx, dtype="float32")

        # --- 1. KHOẢNG CÁCH CƠ BẢN (DISTANCES) ---
        # Vector nối Agent -> Target
        rel_vec = target_ctx.pos - ctx.pos
        dist = np.linalg.norm(rel_vec, axis=1)
        feats["rel_dist"] = pd.Series(dist, index=idx, dtype="float32")

        # --- 2. KHOẢNG CÁCH CHI TIẾT (NOSE-TO-PART) ---
        # Lấy các bộ phận quan trọng
        my_parts = self._extract_parts_dict(ctx, ["nose", "neck"])
        target_parts = self._extract_parts_dict(target_ctx, 
            ["nose", "tail_base", "body_center", "ear_left", "ear_right", 
             "lateral_left", "lateral_right", "tail_midpoint"])

        def dist_ab(pt_a, pt_b):
            if pt_a is None or pt_b is None: return zero()
            d = np.linalg.norm(pt_a - pt_b, axis=1)
            return pd.Series(d, index=idx, dtype="float32")

        an, tn = my_parts["nose"], target_parts["nose"]
        feats["dist_nose_nose"] = dist_ab(an, tn)
        feats["dist_nose_tail"] = dist_ab(an, target_parts["tail_base"])
        feats["dist_nose_body"] = dist_ab(an, target_parts["body_center"])
        feats["dist_nose_el"]   = dist_ab(an, target_parts["ear_left"])
        feats["dist_nose_er"]   = dist_ab(an, target_parts["ear_right"])
        feats["dist_nose_tll"]  = dist_ab(an, target_parts["lateral_left"])
        feats["dist_nose_tlr"]  = dist_ab(an, target_parts["lateral_right"])
        feats["dist_nose_tmp"]  = dist_ab(an, target_parts["tail_midpoint"])

        # --- 3. ĐỊNH HƯỚNG & GÓC NHÌN (ORIENTATION & GAZE) ---
        # Helper lấy vector cơ thể (Mũi - Đuôi/Thân)
        def get_body_vec(parts_dict):
            head = parts_dict.get("nose")
            # Ưu tiên đuôi, nếu ko có thì dùng thân
            tail = parts_dict.get("tail_base")
            if tail is None: tail = parts_dict.get("body_center") # Fallback
            
            if head is not None and tail is not None:
                return head - tail
            return None

        a_vec = get_body_vec(my_parts)
        t_vec = get_body_vec(target_parts)

        # A. Body Cosine: Hai con cùng chiều hay ngược chiều?
        if a_vec is not None and t_vec is not None:
            dot = np.sum(a_vec * t_vec, axis=1)
            mags = np.linalg.norm(a_vec, axis=1) * np.linalg.norm(t_vec, axis=1)
            feats["body_cosine"] = pd.Series(
                np.clip(dot / (mags + 1e-6), -1.0, 1.0), index=idx, dtype="float32"
            )
        else:
            feats["body_cosine"] = zero()

        # B. Gaze Cosine: Tôi có đang nhìn về phía Target không?
        # Vector ánh nhìn = Target_Pos - My_Pos = rel_vec
        if a_vec is not None:
            dot_gaze = np.sum(a_vec * rel_vec, axis=1)
            mag_a = np.linalg.norm(a_vec, axis=1)
            # dist đã tính ở bước 1
            feats["gaze_cosine"] = pd.Series(
                np.clip(dot_gaze / (mag_a * dist + 1e-6), -1.0, 1.0),
                index=idx, dtype="float32"
            )
        else:
            feats["gaze_cosine"] = zero()

        # --- 4. PHÂN RÃ VẬN TỐC (VELOCITY DECOMPOSITION) - CHÌA KHÓA CHO AVOID/ESCAPE ---
        # Vector đơn vị hướng về địch (u)
        dist_safe = dist.copy()
        dist_safe[dist_safe == 0] = 1e-6
        u_vec = rel_vec / dist_safe[:, None]

        # a_vel và t_vel lấy từ Context
        a_vel, t_vel = ctx.vel, target_ctx.vel

        # A. Approach Speed (Vận tốc dọc trục nối 2 con)
        # Dương: Lao vào nhau | Âm: Chạy ra xa nhau
        a_along = np.sum(a_vel * u_vec, axis=1)
        t_along = np.sum(t_vel * (-u_vec), axis=1) # Target hướng ngược lại
        rel_along = np.sum((a_vel - t_vel) * u_vec, axis=1)

        # B. Lateral Speed (Vận tốc ngang - Vuông góc trục nối)
        # Vector chiếu: v_proj = (v . u) * u
        a_proj = a_along[:, None] * u_vec
        a_lat_vec = a_vel - a_proj
        a_lat_speed = np.linalg.norm(a_lat_vec, axis=1)

        feats["approach_speed_agent"]  = pd.Series(a_along, index=idx, dtype="float32")
        feats["approach_speed_target"] = pd.Series(t_along, index=idx, dtype="float32")
        feats["approach_speed_rel"]    = pd.Series(rel_along, index=idx, dtype="float32")
        feats["lateral_speed_agent"]   = pd.Series(a_lat_speed, index=idx, dtype="float32")
        return feats


    # --- Methods tương thích ---
    
    def build_pose_tensor(self, tracking: pd.DataFrame):
        """
        Chuyển dữ liệu tracking (DataFrame) sang Tensor [Frames, Mice, 2] và Dict chi tiết.
        """
        tracking = tracking.sort_values("video_frame")
        frames = np.sort(tracking["video_frame"].unique())
        
        pvid = tracking.pivot(
            index="video_frame", 
            columns=["mouse_id", "bodypart"], 
            values=["x", "y"]
        )
        pvid = pvid.reorder_levels([1, 2, 0], axis=1).sort_index(axis=1).astype("float32")
        mouse_ids = list(pvid.columns.get_level_values(0).unique())
        pos = np.full((len(frames), len(mouse_ids), 2), np.nan, dtype=np.float32)
        per_mouse_df = {}
        
        for i, mid in enumerate(mouse_ids):
            single = pvid[mid]
            per_mouse_df[mid] = single
            
            if "body_center" in single.columns.get_level_values(0):
                cx = single["body_center"]["x"]
                cy = single["body_center"]["y"]
            else:
                cx = single.xs("x", level=1, axis=1).mean(axis=1)
                cy = single.xs("y", level=1, axis=1).mean(axis=1)
            
            pos[:, i, 0] = cx.reindex(frames).values
            pos[:, i, 1] = cy.reindex(frames).values
            
        return frames, mouse_ids, pos, per_mouse_df

    def extract_agent_target(
        self, 
        frames: np.ndarray, 
        mouse_ids: List[Any], 
        pos: np.ndarray, 
        agent_id: Any, 
        target_id: Any, 
        per_mouse_df: Dict = None
    ) -> pd.DataFrame:
        """
        Trích xuất đặc trưng cho cặp (Agent, Target).
        """
        try:
            aid_idx = mouse_ids.index(agent_id)
        except ValueError:
            return pd.DataFrame() 

        # 1. Build Agent Context
        ctx_agent = self._build_context(
            frames, 
            pos[:, aid_idx, :], 
            per_mouse_df.get(agent_id) if per_mouse_df else None
        )

        # 2. Build Target Context
        ctx_target = None
        if self.cfg.use_pairwise and target_id is not None and target_id in mouse_ids:
             tid_idx = mouse_ids.index(target_id)
             ctx_target = self._build_context(
                 frames, 
                 pos[:, tid_idx, :], 
                 per_mouse_df.get(target_id) if per_mouse_df else None
             )

        # 3. Run all features
        all_data = {}
        for func_name, func in self.feature_registry.items():
            out_dict = func(ctx_agent, target_ctx=ctx_target)
            all_data.update(out_dict)

        df_out = pd.DataFrame(all_data, index=ctx_agent.idx)
        df_out = df_out.replace([np.inf, -np.inf], np.nan).fillna(0.0)
        
        return df_out.reindex(sorted(df_out.columns), axis=1)



In [ ]:
# read data
test_dataframe = pl.read_csv(INPUT_DIR / "test.csv")

In [ ]:
# preprocess behavior labels
test_behavior_dataframe = (
    test_dataframe.filter(pl.col("behaviors_labeled").is_not_null())
    .select(
        pl.col("lab_id"),
        pl.col("video_id"),
        pl.col("behaviors_labeled").map_elements(eval, return_dtype=pl.List(pl.Utf8)).alias("behaviors_labeled_list"),
    )
    .explode("behaviors_labeled_list")
    .rename({"behaviors_labeled_list": "behaviors_labeled_element"})
    .select(
        pl.col("lab_id"),
        pl.col("video_id"),
        pl.col("behaviors_labeled_element").str.split(",").list[0].str.replace_all("'", "").alias("agent"),
        pl.col("behaviors_labeled_element").str.split(",").list[1].str.replace_all("'", "").alias("target"),
        pl.col("behaviors_labeled_element").str.split(",").list[2].str.replace_all("'", "").alias("behavior"),
    )
)

test_self_behavior_dataframe = test_behavior_dataframe.filter(pl.col("behavior").is_in(SELF_BEHAVIORS))
test_pair_behavior_dataframe = test_behavior_dataframe.filter(pl.col("behavior").is_in(PAIR_BEHAVIORS))

In [ ]:
%run -i self_features.py
%run -i pair_features.py
%run -i robustify.py

In [ ]:
(WORKING_DIR / "self_features").mkdir(exist_ok=True, parents=True)
(WORKING_DIR / "pair_features").mkdir(exist_ok=True, parents=True)

rows = test_dataframe.rows(named=True)

for row in tqdm(rows, total=len(rows)):
    lab_id = row["lab_id"]
    video_id = row["video_id"]

    tracking_path = TEST_TRACKING_DIR / f"{lab_id}/{video_id}.parquet"
    tracking = pl.read_parquet(tracking_path)

    self_features = make_self_features(metadata=row, tracking=tracking)
    pair_features = make_pair_features(metadata=row, tracking=tracking)

    self_features.write_parquet(WORKING_DIR / "self_features" / f"{video_id}.parquet")
    pair_features.write_parquet(WORKING_DIR / "pair_features" / f"{video_id}.parquet")

    del self_features, pair_features
    gc.collect()

# submissionデータ作成

In [ ]:
# 各グループ（lab_id, video_id, agent, target の組み合わせ）ごとの予測結果を格納するリスト
group_submissions = []

# テストデータを lab_id, video_id, agent, target でグループ化
# maintain_order=True で元の順序を保持
groups = list(test_behavior_dataframe.group_by("lab_id", "video_id", "agent", "target", maintain_order=True))

# 各グループに対して順番に処理を実行（進捗バーを表示）
for (lab_id, video_id, agent, target), group in tqdm(groups, total=len(list(groups))):
    # agent（行動を起こすマウス）のID を抽出
    # 例: "mouse1" → 1
    agent_mouse_id = int(re.search(r"mouse(\d+)", agent).group(1))
    
    # target（行動の対象）のID を抽出
    # "self"（自己行動）の場合は -1、それ以外はマウスIDを抽出
    # 例: "mouse2" → 2, "self" → -1
    target_mouse_id = -1 if target == "self" else int(re.search(r"mouse(\d+)", target).group(1))

    # ===== 特徴量データの読み込み =====
    if target == "self":
        # 自己行動（rear など）の場合: self_features ディレクトリから読み込み
        
        # インデックス列（video_id, agent_mouse_id, video_frame など）を読み込み
        index = (
            pl.scan_parquet(WORKING_DIR / "self_features" / f"{video_id}.parquet")
            .filter((pl.col("agent_mouse_id") == agent_mouse_id))  # 対象マウスでフィルタ
            .select(INDEX_COLS)  # インデックス列のみ選択
            .collect()  # 遅延評価を実行してデータを取得
        )
        
        # 特徴量列（速度、距離、角度など）を読み込み
        feature = (
            pl.scan_parquet(WORKING_DIR / "self_features" / f"{video_id}.parquet")
            .filter((pl.col("agent_mouse_id") == agent_mouse_id))
            .select(pl.exclude(INDEX_COLS))  # インデックス列以外を選択
            .collect()
        )
    else:
        # ペア行動（attack, chase など）の場合: pair_features ディレクトリから読み込み
        
        # インデックス列を読み込み（agent と target の両方でフィルタ）
        index = (
            pl.scan_parquet(WORKING_DIR / "pair_features" / f"{video_id}.parquet")
            .filter((pl.col("agent_mouse_id") == agent_mouse_id) & (pl.col("target_mouse_id") == target_mouse_id))
            .select(INDEX_COLS)
            .collect()
        )
        
        # 特徴量列を読み込み
        feature = (
            pl.scan_parquet(WORKING_DIR / "pair_features" / f"{video_id}.parquet")
            .filter((pl.col("agent_mouse_id") == agent_mouse_id) & (pl.col("target_mouse_id") == target_mouse_id))
            .select(pl.exclude(INDEX_COLS))
            .collect()
        )

    # 予測結果を格納する DataFrame を作成（インデックス列のコピー）
    prediction_dataframe = index.clone()

    # ===== 各行動（behavior）に対して予測を実行 =====
    for row in group.rows(named=True):
        behavior = row["behavior"]  # 現在の行動名（例: "attack", "rear"）

        # 各 fold（交差検証の分割）の予測結果を格納するリスト
        predictions = []  # 予測確率
        prediction_labels = []  # 予測ラベル（閾値で 0/1 に変換したもの）

        # 保存された fold ディレクトリを取得
        # 例: results/AdaptableSnail/attack/fold_0, fold_1, fold_2
        fold_dirs = list((WORKING_DIR / "results" / lab_id / behavior).glob("fold_*"))
        if not fold_dirs:
            # 訓練されたモデルが見つからない場合はスキップ
            continue

        # 各 fold のモデルで予測を実行
        for fold_dir in fold_dirs:
            # 保存された最適閾値を読み込み
            with open(fold_dir / "threshold.txt", "r") as f:
                threshold = float(f.read().strip())
            
            # XGBoost モデルを読み込み
            model = xgb.Booster(model_file=fold_dir / "model.json")
            
            # 特徴量を XGBoost の入力形式（DMatrix）に変換
            dtest = xgb.DMatrix(feature, feature_names=feature.columns)
            
            # モデルで予測を実行（確率値を取得）
            fold_predictions = model.predict(dtest)
            
            # 予測確率を保存
            predictions.append(fold_predictions)
            
            # 閾値を適用してラベル化（1: 行動あり, 0: 行動なし）
            prediction_labels.append((fold_predictions >= threshold).astype(np.int8))

        # 予測結果を DataFrame に追加
        # 各 fold の「予測確率 × 予測ラベル」を列として追加
        # （ラベルが 0 の場合は確率も 0 になり、1 の場合は確率がそのまま残る）
        prediction_dataframe = prediction_dataframe.with_columns(
            *[
                pl.Series(name=f"{behavior}_{fold}", values=predictions[fold] * prediction_labels[fold], dtype=pl.Float32)
                for fold in range(len(fold_dirs))
            ]
        )

    # ===== 最も確率が高い行動を選択 =====
    
    # インデックス列以外の列名を取得（各行動の予測列）
    cols = prediction_dataframe.select(pl.exclude(INDEX_COLS)).columns
    if not cols:
        # 予測列が 1 つもない場合は警告を表示してスキップ
        tqdm.write(f"Warning: No predictions found for {lab_id}, {video_id}, {agent}, {target}")
        continue

    # 各フレームで最も確率が高い行動を選択
    prediction_labels_dataframe = prediction_dataframe.with_columns(
        pl.struct(pl.col(cols))  # 全予測列を構造体にまとめる
        .map_elements(
            # 各行（フレーム）に対して以下の処理を実行:
            # - すべての予測値が 0 なら "none"（行動なし）
            # - それ以外は最大値を持つ行動名を返す
            lambda row: "none" if sum(row.values()) == 0 else (cols[np.argmax(list(row.values()))]).split("_")[0],
            return_dtype=pl.String,
        )
        .alias("prediction")  # 新しい列名を "prediction" とする
    ).select(INDEX_COLS + ["prediction"])  # インデックス列と予測列のみ選択

    # ===== 連続する同じ行動をイベントにまとめる =====
    
    group_submission = (
        prediction_labels_dataframe
        .filter((pl.col("prediction") != pl.col("prediction").shift(1)))  # 行動が変化したフレームのみ残す
        .with_columns(pl.col("video_frame").shift(-1).alias("stop_frame"))  # 次の変化点を終了フレームとする
        .filter(pl.col("prediction") != "none")  # "none"（行動なし）を除外
        .select(
            # 提出形式に合わせて列を選択・変換
            pl.col("video_id"),
            ("mouse" + pl.col("agent_mouse_id").cast(str)).alias("agent_id"),  # 例: 1 → "mouse1"
            pl.when(pl.col("target_mouse_id") == -1)  # target_mouse_id が -1 なら
            .then(pl.lit("self"))  # "self" に変換
            .otherwise("mouse" + pl.col("target_mouse_id").cast(str))  # それ以外は "mouseN"
            .alias("target_id"),
            pl.col("prediction").alias("action"),  # 行動名
            pl.col("video_frame").alias("start_frame"),  # 開始フレーム
            pl.col("stop_frame"),  # 終了フレーム
        )
    )

    # このグループの提出データをリストに追加
    group_submissions.append(group_submission)

# ===== 全グループの予測結果を結合 =====

# 全グループの提出データを縦方向に結合
submission = pl.concat(group_submissions, how="vertical").sort(
    "video_id",
    "agent_id",
    "target_id",
    "action",
    "start_frame",
    "stop_frame",
)

# 提出データの堅牢化処理（重複削除、フレームの修正など）
submission = robustify(submission, test_dataframe, train_test="test")

# 行番号（row_id）を追加して CSV ファイルとして保存
submission.with_row_index("row_id").write_csv(WORKING_DIR / "submission.csv")

In [ ]:
!head submission.csv